In [4]:
# Core libraries
import pandas as pd
import polars as pl

# Internal data access
from op_analytics.coreutils.duckdb_inmem.client import init_client, register_parquet_relation
from op_analytics.coreutils.clickhouse.oplabs import run_query_oplabs

# configurations
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import config

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
CHAIN = config.CHAIN
BLOCK_DATE = config.BLOCK_DATE

In [9]:
parquet_paths = [
    f"gs://oplabs-tools-data-sink/blockbatch/refined_traces/refined_transactions_fees_v2/chain=base/dt={BLOCK_DATE}/*.parquet"
]

# Load into DuckDB
ctx = init_client()
block_view = register_parquet_relation("refined_transactions_fees_v2", parquet_paths)

2025-11-13 16:33:03 [info     ] registered view: 'refined_transactions_fees_v2' using 1 parquet paths filename=client.py lineno=319 process=13359


In [10]:
# Preview data 
df_sample = ctx.client.sql(f"SELECT * FROM {block_view} LIMIT 10").pl()

print(df_sample)

df_sample.schema

shape: (10, 59)
┌──────────┬─────────┬──────────┬─────────────┬───┬─────────────┬─────────────┬───────┬────────────┐
│ chain_id ┆ network ┆ nonce    ┆ transaction ┆ … ┆ l1_base_gas ┆ l1_blob_bas ┆ chain ┆ dt         │
│ ---      ┆ ---     ┆ ---      ┆ _index      ┆   ┆ _price_gwei ┆ e_gas_price ┆ ---   ┆ ---        │
│ i32      ┆ str     ┆ i64      ┆ ---         ┆   ┆ ---         ┆ _gwei       ┆ str   ┆ date       │
│          ┆         ┆          ┆ i64         ┆   ┆ decimal[38, ┆ ---         ┆       ┆            │
│          ┆         ┆          ┆             ┆   ┆ 10]         ┆ decimal[38, ┆       ┆            │
│          ┆         ┆          ┆             ┆   ┆             ┆ 10]         ┆       ┆            │
╞══════════╪═════════╪══════════╪═════════════╪═══╪═════════════╪═════════════╪═══════╪════════════╡
│ 8453     ┆ mainnet ┆ 32917387 ┆ 0           ┆ … ┆ null        ┆ null        ┆ base  ┆ 2025-07-16 │
│ 8453     ┆ mainnet ┆ 3171     ┆ 1           ┆ … ┆ 1.198867975 ┆ 0.0000000

Schema([('chain_id', Int32),
        ('network', String),
        ('nonce', Int64),
        ('transaction_index', Int64),
        ('from_address', String),
        ('to_address', String),
        ('block_number', Int64),
        ('block_timestamp', UInt32),
        ('hash', String),
        ('transaction_type', Int32),
        ('gas_price', Int64),
        ('gas_limit', Int64),
        ('l2_gas_used', Int64),
        ('receipt_l1_gas_used', Int64),
        ('l1_fee', Int64),
        ('l1_gas_price', Int64),
        ('l1_blob_base_fee', Int64),
        ('base_fee_per_gas', Int64),
        ('max_priority_fee_per_gas', Int64),
        ('effective_l2_priority_fee_per_gas', Int64),
        ('l1_fee_scalar', Decimal(precision=12, scale=6)),
        ('l1_base_fee_scalar', Decimal(precision=26, scale=7)),
        ('l1_blob_base_fee_scalar', Decimal(precision=26, scale=7)),
        ('legacy_extra_fee_per_gas', Int64),
        ('l2_fee', Int64),
        ('l2_priority_fee', Int64),
        ('l2_b